In [2]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [7]:
from runner import *
from process_results import *
import os
import pandas as pd
fig_dir = "/usr0/home/aoli/repos/exchain-paper/figs"

In [23]:
table_data = []

for name, app in TEST_APPLICATIONS.items():
    table_data.append([])
    table_data[-1].append(name.replace("_", "-").upper())
    ground_truth = app.read_ground_truth()
    dynamic_dependency = app.read_latest_dynamic_dependency()
    root_cause_in_log = check_root_cause_in_log(ground_truth, app.origin_log_path)
    table_data[-1].append(app.is_running_service)
    table_data[-1].append(app.is_async)
    table_data[-1].append(root_cause_in_log)

    table_data[-1].append(get_exception_distance(ground_truth,
                                                   os.path.join(app.get_latest_result("dynamic"), "exception.json")))


    dynamic_result_path = app.get_latest_result("dynamic")
    exceptions = read_exceptions(os.path.join(dynamic_result_path, "exception.json"))
    table_data[-1].append(len(exceptions))
    ground_truth = app.read_ground_truth()
    dynamic_dependency = app.read_latest_dynamic_dependency()


    static_dependency = app.read_latest_static_dependency()
    table_data[-1].extend(process_dependency_result(static_dependency, ground_truth))

    hybrid_online, hybrid_offline = app.read_latest_hybrid_dependency()
    table_data[-1].extend(process_dependency_result(hybrid_online + hybrid_offline, ground_truth))
    table_data[-1].extend(process_dependency_result(dynamic_dependency, ground_truth))
save_as_latex_table(table_data, os.path.join(fig_dir, "issue_result.tex"))

In [4]:

table_data = []

for name, app in BENCH_APPLICATIONS.items():
    table_data.append([])
    table_data[-1].append(name.replace("_", "-").upper())
    separate_results = ["fineract_bench", "jena_bench"]
    print(name)
    if "jena" in name:
        continue
    table_data.append(read_perf_result(app))
save_as_latex_table(table_data, os.path.join(fig_dir, "perf_result.tex"))



fineract_bench
jena_bench
hadoop_nnbench
wicket_bench
solr_bench
hadoop_terasort
tomcat_bench
hadoop_dfs_write
               name   origin    std   static    std   diff   hybrid    std  \
0    fineract_bench     12.8    0.1     13.5    0.1   6.0%     15.9    0.1   
1    hadoop_nnbench   1535.4    4.0   1516.2    3.8  -1.3%   1209.8    2.6   
2      wicket_bench     25.8    1.1     26.4    0.8   2.3%     30.2    0.7   
3        solr_bench  16581.6  158.1  16138.1  110.4  -2.7%  12079.3  114.9   
4   hadoop_terasort     20.6    0.1     22.7    0.1  10.3%     26.8    0.0   
5      tomcat_bench      8.2    0.2      8.7    0.4   6.4%     10.6    0.5   
6  hadoop_dfs_write   2094.4   28.2   2093.5   14.3  -0.0%   1797.1   18.0   

     diff dynamic   std     diff  
0   24.9%   206.1  17.9  1514.5%  
1  -21.2%   687.5   2.0   -55.2%  
2   17.1%   452.5  16.4  1653.5%  
3  -27.2%   219.4  18.2   -98.7%  
4   30.2%  1257.4  55.6  6004.6%  
5   30.4%   143.9   6.2  1662.0%  
6  -14.2%   323.0  